# Initial analysis of AO files

In [ ]:
import logging

import numpy as np
print(f"numpy version {np.__version__}")
import matplotlib
import matplotlib.pyplot as plt
print(f"matplotlib version {matplotlib.__version__}")
import pandas as pd
print(f"pandas version {pd.__version__}")
import scipy.signal as sig
import scipy.io as sio
from scipy import __version__ as sciver
print(f"scipy version {sciver}")

# Enable reloading of packages upon changes
%reload_ext autoreload
%autoreload 2

# Enable resizing of Jupyter notebook based on the browser width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(asctime)s: %(message)s')

## Parameters

In [ ]:
sampRate = 44000

fileDir = 'X:\\Katya\\K4\\15-12-2019\\'
#fileDir = 'C:\\IBG\\data\\Katya\\29-12-2019\\'
filePrefix = 'F191215-'

fileList = list(range(6,24))
#fileList = list(range(5,18))
logging.info(f'Files: {fileList}')

#elecList = [8, 10, 12, 14, 18, 19, 22, 25, 31]
#elecList = [2,11,13,18,19,21,22,23,24,26,29,30,31,32]
elecList = list(range(1,33))

#goodElecList = [2,11,13,18,19,21,22,23,24,26,29,30,31,32]
goodElecList = [2,5,7,11,13,18,19,22,23,26,27,29,31]
#badElecList = [1,20,21,25]
badElecList = []
refElecList = set(elecList) - set(goodElecList) - set(badElecList)

def splitList(elecList):
    lowList = [x for x in elecList if x<17 ]
    highList = [x for x in elecList if x>16 ]
    return lowList, highList

lowGoodList, highGoodList = splitList(goodElecList)
lowRefList, highRefList = splitList(goodElecList)

logging.info(f'electrodes: {elecList}')

bPass, aPass = sig.butter(4, [300/(sampRate/2),8000/(sampRate/2)], btype='bandpass')
bNotch, aNotch = sig.iirnotch(50/(1000/2), 30)

## Convert files

In [ ]:
for elecNum in elecList:
    elecName = f'CRAW_{elecNum:03d}'
    logging.info(f'Processing electrode: {elecName}')
    outRawFileName = f'{fileDir}{filePrefix}Raw{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    outFilterFileName = f'{fileDir}{filePrefix}Filter{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    outLfpFileName = f'{fileDir}{filePrefix}Lfp{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    elecData = [None] * len(fileList)
    for i, fileNum in enumerate(fileList):
        fileName = f'{fileDir}{filePrefix}{fileNum:04d}.mat'
        logging.info(f'Processing electrode: {fileName}')
        matList=sio.loadmat(fileName, variable_names=elecName)
        elecData[i] = matList[elecName][0,:]
    allData = np.concatenate(elecData)
    allData.tofile(outRawFileName)
    filtData = sig.filtfilt(bPass, aPass, allData)
    filtData.astype('int16').tofile(outFilterFileName)
    lfpData = sig.decimate(allData, int(sampRate/1000), ftype='fir')
    lfpData = sig.filtfilt(bNotch, aNotch, lfpData)
    lfpData.astype('int16').tofile(outLfpFileName)

In [ ]:
def createRef(elecList,outFile,batchSize=1000000):
    ifids = []
    ofid = open(outFile,'w')
    for elecNum in elecList:
        filterFileName = f'{fileDir}{filePrefix}Filter{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
        print(outFile)
        print(filterFileName)
        try:
            ifids.append(open(filterFileName, 'r'))
        except OSError:
            logging.error(f'Cannot open file {filterFileName}')
            return 
    location, readMore = 0, True
    inBuffer = np.zeros((len(elecList), batchSize), dtype=np.int16)
    while readMore:
        logging.info(f'Reading location {location}.')
        for i, elec in enumerate(elecList):
            data = np.fromfile(ifids[i], count=batchSize, dtype=np.int16)
            if i == 0 and data.shape[0] != batchSize:
                inBuffer = np.zeros((len(elecList), data.shape[0]), dtype=np.int16)
                readMore = False
            inBuffer[i, :] = data
        medianBuffer = np.median(inBuffer, axis=0)
        medianBuffer.astype(np.int16).tofile(ofid)
        location += data.shape[0]
    logging.info(f'Finished writing {location} samepls to reference file.')
    ofid.close()
    for ifid in ifids:
        ifid.close()

def removeRef(elecList,refFile,batchSize=1000000):
    ifids, ofids = [], []
    rfid = open(refFile,'r')
    for elecNum in elecList:
        inFileName = f'{fileDir}{filePrefix}Filter{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
        outFileName = f'{fileDir}{filePrefix}FilterRef{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
        try:
            ifids.append(open(inFileName, 'r'))
            ofids.append(open(outFileName, 'w'))
        except OSError:
            logging.error(f'Cannot open file {inFileName} {outFileName}')
            return 
    location, readMore = 0, True
    while readMore:
        refBuffer = np.fromfile(rfid, count=batchSize, dtype=np.int16)
        logging.info(f'Reading location {location}.')
        for i, elec in enumerate(elecList):
            buffer = np.fromfile(ifids[i], count=batchSize, dtype=np.int16)
            if i == 0 and buffer.shape[0] != batchSize:
                readMore = False
            buffer = buffer - refBuffer
            buffer.astype(np.int16).tofile(ofids[i])
        location += buffer.shape[0]
    logging.info(f'Finished writing {location} samples to reference file.')
    rfid.close()
    for ifid in ifids:
        ifid.close()
    for ofid in ifids:
        ofid.close()
 
createRef(lowRefList,f'{fileDir}{filePrefix}FilterLowRef-{fileList[0]}-{fileList[-1]}.bin' )
removeRef(lowGoodList,f'{fileDir}{filePrefix}FilterLowRef-{fileList[0]}-{fileList[-1]}.bin' )

createRef(highRefList,f'{fileDir}{filePrefix}FilterHighRef-{fileList[0]}-{fileList[-1]}.bin' )
removeRef(highGoodList,f'{fileDir}{filePrefix}FilterHighRef-{fileList[0]}-{fileList[-1]}.bin' )


In [ ]:
samples=10000000

rawData = [None] * 33
lfpData = [None] * 33
filterData = [None] * 33
cleanData = [None] * 33
refData = [None] *2

for elecNum in goodElecList:
    rawFileName = f'{fileDir}{filePrefix}Raw{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    rawData[elecNum] = np.fromfile(rawFileName,dtype=np.int16, count=samples)
    filterFileName = f'{fileDir}{filePrefix}Filter{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    filterData[elecNum] = np.fromfile(filterFileName,dtype=np.int16, count=samples)
    cleanFileName = f'{fileDir}{filePrefix}FilterRef{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    cleanData[elecNum] = np.fromfile(cleanFileName,dtype=np.int16, count=samples)
    lfpFileName = f'{fileDir}{filePrefix}Lfp{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    lfpData[elecNum] = np.fromfile(lfpFileName,dtype=np.int16)

lowRefFileName = f'{fileDir}{filePrefix}FilterLowRef-{fileList[0]}-{fileList[-1]}.bin' 
refData[0] = np.fromfile(lowRefFileName,dtype=np.int16)
highRefFileName = f'{fileDir}{filePrefix}FilterHighRef-{fileList[0]}-{fileList[-1]}.bin' 
refData[1] = np.fromfile(highRefFileName,dtype=np.int16)

In [ ]:
%matplotlib widget                 
matplotlib.rcParams['figure.figsize'] = [30, 40]
b=6500000
e=b+44000*10
plotDiff = 2000
#plt.plot(rawData[0][b:e],'r', linewidth=0.5)
plt.plot(refData[0][b:e]-1*plotDiff,'g', linewidth=0.5)
plt.plot(refData[1][b:e]-2*plotDiff,'b', linewidth=0.5)
for i,elecNum in enumerate(goodElecList):
    plt.plot(filterData[elecNum][b:e]-(2*i+3)*2000,linewidth=0.5)
    plt.plot(cleanData[elecNum][b:e]-(2*i+4)*2000,linewidth=0.5)
plt.legend()

In [ ]:
cleanData = sig.filtfilt(bNotch, aNotch, lfpData)
f, Pxx_den = sig.welch(cleanData, 1000, nperseg=1024)
plt.semilogy(f, Pxx_den)

In [ ]:
samples=10000000

filterData = [None] * 33

for elecNum in elecList:
    filterFileName = f'{fileDir}{filePrefix}Filter{elecNum:03d}-{fileList[0]}-{fileList[-1]}.bin' 
    filterData[elecNum] = np.fromfile(filterFileName,dtype=np.int16, count=samples)

In [ ]:
a=np.corrcoef([filterData[elecNum] for elecNum in elecList])
%matplotlib widget      
matplotlib.rcParams['figure.figsize'] = [30, 10]
plt.imshow(a)
plt.colorbar()
